# Binomial Pricing Model

In [1]:
import numpy as np 
from math import exp, sqrt

In [2]:

class BinomialPriceModel(object):
    def __init__(self,BPM_Paramaters):
        self.So, self.h = BPM_Paramaters["So"], BPM_Paramaters["h"]
        self.k, self.sig = BPM_Paramaters["k"], BPM_Paramaters["sig"]
        self.r, self.delta = BPM_Paramaters["r"], BPM_Paramaters["delta"]
        
    def solve(self,option,flavor):      
        # Gather u and d values at each step
        u_d = []
        [u_d.append([i,self.u_value(h),self.d_value(h),h]) for i,h in\
                 enumerate(self.h[::-1])]
        
        # Gather values at each step
        s_values = np.zeros((len(u_d)+1, len(u_d)+1),dtype="float64")
        s_values[0,0] = self.So
        for i in range(1,len(u_d)+1):
            s_values[0,i] = u_d[i-1][1]*s_values[0,i-1]
            for j in range(1,i+1):
                s_values[j,i] = u_d[i-1][2]*s_values[j-1,i-1]     
                
        # Get European Values
        vo_values = np.zeros((len(u_d)+1, len(u_d)+1),dtype="float64")
        
        for i in range(len(u_d)+1):
            if option == "Call":
                vo_values[i,len(u_d)] = max(0, s_values[i,len(u_d)]-self.k)
            elif option == "Put":  
                vo_values[i,len(u_d)] = max(0,self.k-s_values[i,len(u_d)])
                       
        v_values = vo_values   
        for t in list(reversed(range(len(u_d)))):
            for i in range(t+1):
                a = self.Pu_value(u_d[i][1],u_d[i][2],u_d[i][3])*vo_values[i,t+1]
                b = self.Pd_value(u_d[i][1],u_d[i][2],u_d[i][3])*vo_values[i+1,t+1]
                v_values[i,t] = a+b
                
        # Get American values 
        if flavor == "A": 
            Av_values = vo_values
            Av_bool_values = np.zeros((len(u_d)+1, len(u_d)+1),dtype="float64")
            
            for t in list(reversed(range(len(u_d)))):
                for i in list(range(t+1)):
                    if option == "Put":
                        e = self.k - s_values[i,t]
                    if option == "Call":
                        e = s_values[i,t] - self.k
                        
                    a = self.Pu_value(u_d[i][1],u_d[i][2],u_d[i][3])*vo_values[i,t+1]
                    b = self.Pd_value(u_d[i][1],u_d[i][2],u_d[i][3])*vo_values[i+1,t+1]
                    x = a+b
                    Av_values[i,t] = max(e,x)
                    if e > x: 
                        Av_bool_values[i,t] = 1
                        
            mask = Av_values[:,-1] != 0  
            Av_bool_values[:,-1] = mask
        
        if flavor == "E":
            asset_value = v_values[0,0] 
            return asset_value
        
        if flavor == "A": 
            return (Av_values,Av_bool_values)
            
    def u_value(self,h):
        return exp((self.r-self.delta)*h + self.sig*sqrt(h))

    def d_value(self,h):
        return exp((self.r-self.delta)*h - self.sig*sqrt(h))
    
    def Pu_value(self,u,d,h):
        return exp(-self.r*h)*((exp((self.r-self.delta)*h)-d)/(u-d))
     
    def Pd_value(self,u,d,h):
        return exp(-self.r*h)*((u-exp((self.r-self.delta)*h))/(u-d))    
    

In [3]:
n = 9

BPM_Paramaters = {
"So": 100,
"h": [1/n]*n, 
"k": 80, 
"sig": .4,
"r": 0,
"delta": 0.01,      
}

model = BinomialPriceModel(BPM_Paramaters)

In [4]:
EC_price = model.solve(option="Call",flavor="E")
print("European Call Price: ", np.round(EC_price,8))

European Call Price:  25.79232528


In [5]:
EP_price = model.solve(option="Put",flavor="E")
print("European Put Price: ",np.round(EP_price,8))

European Put Price:  6.78734191


In [6]:
AP_price, AP_bool = model.solve(option="Put",flavor="A")
np.set_printoptions(precision=3)
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
print("American Put Price: ","\n",np.round(AP_price,8))

American Put Price:  
 [[ 6.787  3.517  1.382  0.314  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  9.649  5.386  2.318  0.588  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  13.380  8.071  3.832  1.103  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  18.027  11.781  6.220  2.068  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  23.494  16.648  9.853  3.878  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  29.485  22.595  15.083  7.271
   0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  35.514  29.169  21.919
   13.635]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  41.067  35.514
   29.169]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  45.927  41.067]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  50.180]]


In [7]:
print(AP_bool)

[[ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  1.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  1.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  1.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  1.000]]


In [9]:
AC_price, AC_bool = model.solve(option="Call",flavor="A")
print("American Call Price: ","\n",np.round(AC_price,8))
np.set_printoptions(precision=3)
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

American Call Price:  
 [[ 26.013  36.973  51.125  68.694  89.705  113.694  141.075  172.327
   207.996  248.708]
 [ 0.000  16.420  24.588  35.749  50.305  68.356  89.328  113.264
   140.585  171.767]
 [ 0.000  0.000  9.272  14.821  23.009  34.508  49.693  68.027  88.952
   112.835]
 [ 0.000  0.000  0.000  4.415  7.655  12.947  21.218  33.378  49.405
   67.698]
 [ 0.000  0.000  0.000  0.000  1.580  3.024  5.708  10.576  19.115
   33.126]
 [ 0.000  0.000  0.000  0.000  0.000  0.315  0.676  1.448  3.102  6.647]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]]


In [10]:
print(AC_bool)

[[ 0.000  0.000  0.000  0.000  1.000  1.000  1.000  1.000  1.000  1.000]
 [ 0.000  0.000  0.000  0.000  0.000  1.000  1.000  1.000  1.000  1.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  1.000  1.000  1.000  1.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  1.000  1.000  1.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  1.000  1.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  1.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]]
